In [1]:
from __future__ import division, print_function
%load_ext autoreload
%autoreload 2

In [4]:
import os.path

import numpy as np
import scipy.io

from irispy import Polyhedron

from director import viewerclient as vc

from pydrake.parsers import PackageMap
from pydrake import getDrakePath
from pydrake import rbtree
from pydrake.solvers import ik
from pydrake.trajectories import PiecewisePolynomial
from pydrake.lcm import DrakeLcm
from pydrake.systems import DrakeVisualizer

import mpc_tools.mpcqp as mqp
from boxatlas import boxatlas as box
from boxatlas.contactstabilization import BoxAtlasContactStabilization

In [5]:
vis = vc.Visualizer()

In [6]:
surfaces = [
    box.Surface(Polyhedron.fromBounds([1.2, 0], [1.2, 2]),
            Polyhedron(np.array([[1, -1], [1, 1]]), np.array([0, 0]))),
    box.Surface(Polyhedron.fromBounds([-2, 0], [2, 0]), 
            Polyhedron(np.array([[-2, -1], [2, -1]]), np.array([0, 0]))),
    box.Surface(Polyhedron.fromBounds([-2, 0], [2, 0]), 
            Polyhedron(np.array([[-2, -1], [2, -1]]), np.array([0, 0]))),
    box.Surface(Polyhedron.fromBounds([-1.2, 0], [-1.2, 2]),
            Polyhedron(np.array([[-1, -1], [-1, 1]]), np.array([0, 0]))),
]

env = box.Environment(surfaces, Polyhedron.fromBounds([-1.2, 0], [1.2, 2]))
atlas = box.BoxAtlas()
initial_state = box.BoxAtlasState(atlas)
initial_state.qcom = np.array([0, 1])
initial_state.vcom = np.array([-4, 0.])
initial_state.qlimb = map(np.array, [[0.75, 1], [0.25, 0], [-0.25, 0], [-0.75, 1]])

opt = BoxAtlasContactStabilization(initial_state, env, num_time_steps=2)
solnData = opt.solve()

SolutionResult.kSolutionFound


In [15]:
box.planPlayback(vis, solnData)

In [13]:
solnData.states.components[0].breaks

array([ 0.  ,  0.05,  0.1 ])

In [10]:
solnData.states(0).vcom

array([-4.,  0.])